# **Exercise session 2: Convolutional neural network to detect NO$_2$ emissions from satellite data**

In this exercise session you will train a convolutional neural network (CNN) to detect NO$_2$ plumes from Sentinel-5P TROPOMI instrument. This exercise is designed to be completed on Aalto JupyterHub. Please ensure that your notebook includes all necessary installation commands for any additional libraries your code requires. These commands should be clearly written and integrated within your notebook.

In [10]:
# Importing necessary libraries

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio  # For handling raster data
import geopandas as gpd  # For handling vector data

from sklearn.model_selection import train_test_split
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cpu


## **Exercise 2.1: Data preparation**

In the previous exercise session, you built a small dataset of 20 images containing a plume and 20 images containing no plume. These images have been combined with those from your classmates to form a larger dataset, suitable for training a convolutional neural network (CNN). Your task now is to prepare this dataset for the training process.
1. The pixel values in your images should be normalized to have values between 0 and 1. This is a common practice in image processing for neural networks, as it helps with the convergence during training.
2. Separate the data into training, validation and test sets:
   - **Training set:** Used for training the CNN. Typically, this is the largest portion of the data.
   - **Validation set:** Used to tune the hyperparameters and evaluate the models during training.
   - **Test set:** Used to test the model after the training process is complete. This set should only be used once to evaluate the final model performance.

A common split ratio is 70% training, 15% validation, and 15% test. However, the exact split may depend on the size of your dataset.

In [11]:
import os
import numpy as np

print("Current working directory", os.getcwd())
print()
# Define the directories
plume_dir = '../../../coursedata/plumes'
n_plume_dir = '../../../coursedata/n-plumes'

# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors and scale to [0, 1]
    transforms.Normalize((0.5,), (0.5,))  # Normalize to have mean=0.5 and std=0.5 (adjust if necessary)
])

# Load images using ImageFolder
dataset = datasets.ImageFolder(root='coursedata',
                               transform=transform)

# Splitting the dataset into train, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - (train_size + val_size)

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders for each set
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Check the sizes
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")


Current working directory /notebooks/mlca2024/exercises-week-2



FileNotFoundError: [Errno 2] No such file or directory: 'coursedata'

## **Exercise 2.2: Build a convolutional neural network model**

Build a basic CNN model using a framework like TensorFlow or PyTorch. You can use [the code from this tutorial](https://www.tensorflow.org/tutorials/images/cnn) for constructing a simple CNN. The model doesn't need to be very deep; a few convolutional layers should be enough. Explain your model construction in your own words.

Model structure guidelines (that you can ignore):
1. **Input layer:** Your input layer should match the dimensions of your plume images.
2. **Convolutional layers:**
   - Start with 2-3 convolutional layers.
   - Use small filters/kernels (like 3x3 or 5x5) to capture the image features.
   - Apply activation functions like ReLU to introduce non-linearity.
3. **Pooling layers:** After each convolutional layer, add a pooling layer (like max pooling) to reduce the spatial dimensions of the output volume.
4. **Flatten layer:** Flatten the output from the convolutional layers before feeding it into the fully connected layers.
5. **Fully connected (dense) layers:**
   - A couple of dense layers at the end of the network for classification.
   - The last dense layer should have the number of units equal to the number of classes and use a softmax activation function for multi-class classification.
6. **Compile the model:**
   - Compile the model with an appropriate optimizer (like Adam or SGD).
   - Choose a suitable loss function (like binary_crossentropy for binary classification).
   - Include accuracy as a metric for model performance evaluation.

## **Exercise 2.3: Train the model**

1. Use the training and validation data to train the model.
2. Monitor the loss and accuracy metrics over epochs to observe the training process.
3. Save the model (optional): After training, you can save your model for future use.

Things to Monitor:
- **Overfitting:** If your model performs well on training data but poorly on validation data, it might be overfitting. Consider using dropout layers or getting more data.
- **Underfitting:** If the model performs poorly on both training and validation data, it might be underfitting. Consider adding complexity to your model or training for more epochs.

## **Exercise 2.4: Model evaluation**

After training your model, the next step is to evaluate its performance using the test set. This step is crucial to understand how well your model will perform on unseen data. Some common metrics to evaluate a CNN that performs a classification task are
- **Accuracy:** Measures the percentage of correctly predicted instances out of all predictions. It's straightforward but can be misleading in cases of class imbalance.
- **Precision** (Positive predictive value): The ratio of correctly predicted positive observations to the total predicted positives. It's important when the cost of False Positives is high.
- **Recall** (sensitivity, true positive rate): The ratio of correctly predicted positive observations to all actual positives. Crucial when the cost of false negatives is high.
- **F1 score:** The weighted average of precision and recall. Useful as a single metric when you need a balance between precision and recall.
- **Confusion matrix:** A table layout that allows visualization of the performance of an algorithm. Shows the number of true positives, false positives, true negatives, and false negatives.

## **Exercise 2.5: Visualization of results**

Visualize model predictions: Choose a subset of your test dataset and display the images with their predicted and actual labels. This is particularly useful to see how the model performs on individual examples.

## **Exercise 2.6: Test your model on the real world** (Optional)

1. Pick an area of roughly 500 000 km$^2$ that includes industrial activity, and consider a period of around one month to ensure enough data. Use Copernicus Browser for a preliminar analysis of a good area.
2. Retrieve Sentinel-5P NO$_2$ data from that area and that time period.
3. Prepare the data to fit your CNN. The images you feed your CNN need to be of size 64x64 pixels. Make sure you retain the coordinates of your data to be able to locate potential power plants or cities later.
4. Look for plumes using your trained CNN.
5. How many plumes can you find? Can you link them to any known city or power plant?

**Note:** You will need to identify cloud covered-areas and exclude them from the dataset. You can also interpolate missing values due to clouds if there are only a few missing values.

## **Exercise 2.7: Overview**

Reflect on the entire process you've undertaken, from the initial data collection to building and training your machine learning algorithm, and finally evaluating this algorithm in a real-world setting. Which step did you find the most challenging, and why? Additionally, were there any unexpected difficulties or learning experiences along the way? How do you think these challenges could be addressed in future projects? (Imagine this was a task in your job, this is not a question on how to improve homework exercises!)

## **Exercise 2.8: Comparison**
Compare your approach for detecting NO$_2$ plumes with the method described in the class-reviewed paper [Automated detection of atmospheric NO$_2$ plumes from satellite data: a tool to help infer anthropogenic combustion emissions.](https://amt.copernicus.org/articles/15/721/2022/). Focus on key differences and similarities in the data processing techniques, CNN architecture, and overall detection strategy. Briefly discuss how these differences might impact the effectiveness of plume detection

Discuss how using a plume detection algorithm based on a CNN model can be utilized for climate action. Interpret the results, noting any interesting observations, and suggest potential areas for model improvement. Could other approaches be better for such a task? What steps would you take next?